Please download the data from L2 Artic site

In [ ]:
import glob
import os
import random
import unittest
import shutil
random.seed(10)
input_folder = r"C:\Users\isabe\Downloads\l2 artic dataset"

Check if all the folders have the same files that are manually annotated
and the same files that are not manually annotated
If this is true, it is easier to split

In [ ]:
# check if all manual annotation filenames are the same
folders = glob.glob(os.path.join(input_folder, "*"))
sample_folder = folders[0]
print("Sample folder: ", sample_folder)
sample_manual_annotations = glob.glob(os.path.join(sample_folder, "annotation", "*.TextGrid"))
sample_manual_annotations = [path.split("\\")[-1][:-9] for path in sample_manual_annotations]
existing_manual_annotation_filenames = set(sample_manual_annotations)
for folder in folders[1:]:
    print("Comparison folder: ", folder)
    manual_annotations = glob.glob(os.path.join(folder, "annotation", "*.TextGrid"))
    manual_annotations = [path.split("\\")[-1][:-9]  for path in manual_annotations]
    unique_manual_annotation_filenames = set(manual_annotations)
    print(f"\nIn {folder} but not in {sample_folder}")
    print(unique_manual_annotation_filenames.difference(existing_manual_annotation_filenames))
    print(f"\nIn {sample_folder} but not in {folder}")
    print(existing_manual_annotation_filenames.difference(unique_manual_annotation_filenames))
    assert existing_manual_annotation_filenames == unique_manual_annotation_filenames, "There are different files annotated for different speakers"

print("Manual annotation filenames are all the same!")


Sample folder:  C:\Users\isabe\Downloads\l2 artic dataset\BWC
Comparison folder:  C:\Users\isabe\Downloads\l2 artic dataset\HJK

In C:\Users\isabe\Downloads\l2 artic dataset\HJK but not in C:\Users\isabe\Downloads\l2 artic dataset\BWC
{'arctic_a0148', 'arctic_a0233', 'arctic_b0469', 'arctic_b0019', 'arctic_a0384', 'arctic_b0192', 'arctic_b0047', 'arctic_b0068', 'arctic_a0143', 'arctic_a0379', 'arctic_a0145', 'arctic_b0238', 'arctic_b0355', 'arctic_b0425', 'arctic_b0304', 'arctic_a0375', 'arctic_b0518', 'arctic_a0378', 'arctic_a0542', 'arctic_a0168', 'arctic_a0416', 'arctic_b0215', 'arctic_b0315', 'arctic_a0144', 'arctic_a0442', 'arctic_a0475', 'arctic_a0271', 'arctic_b0266', 'arctic_a0178', 'arctic_b0503', 'arctic_b0197', 'arctic_b0225', 'arctic_b0342', 'arctic_a0258', 'arctic_a0364', 'arctic_b0143', 'arctic_b0386', 'arctic_a0369', 'arctic_b0022', 'arctic_a0303', 'arctic_a0260', 'arctic_b0320', 'arctic_b0201', 'arctic_b0106', 'arctic_b0530', 'arctic_a0345', 'arctic_a0272', 'arctic_a020

AssertionError: There are different files annotated for different speakers

Note that the total number of files per folder can differ because some speakers
did not speak some audios / due to recording issues so audio was omitted

Obtain filenames in test and filenames in train for each folder to save
to a dictionary with key being folder path and value being {"test": [...], "train":[...]

In [ ]:
input_folder = r"C:\Users\isabe\Downloads\l2 artic dataset"
folder_to_filename_dict = {}

for folder in glob.glob(os.path.join(input_folder, "*")):
    print(folder)
    output_folder = folder.replace("l2 artic dataset", "l2 artic dataset split")

    # splits file with manual annotations first
    manual_annotations = glob.glob(os.path.join(folder, "annotation", "*.TextGrid"))
    print(f"Total number of files with manual annotations: {len(manual_annotations)}")
    manual_annotations = [path.split("\\")[-1][:-9] for path in manual_annotations]
    test_manual = random.sample(manual_annotations, k=round(len(manual_annotations) * 0.2))
    train_manual = [path for path in manual_annotations if path not in test_manual]
    print(f"{len(test_manual)} files with manual annotation in test and {len(train_manual)} in train")

    # split files without manual annotations
    wav = glob.glob(os.path.join(folder, "wav", "*.wav"))
    all_filenames = [file.split("\\")[-1][:-4] for file in wav]
    print(f"Total number of files: {len(all_filenames)}")
    files_without_manual_annotation = [file for file in all_filenames if file not in manual_annotations]
    print(f"Total number of files without manual annotations: {len(files_without_manual_annotation)}")
    test_without_manual = random.sample(files_without_manual_annotation, k=round(len(files_without_manual_annotation) * 0.2))
    train_without_manual = [path for path in files_without_manual_annotation if path not in test_without_manual]
    print(f"{len(test_without_manual)} files with no manual annotation in test and {len(train_without_manual)} in train")

    # combine to form test and train
    test = test_manual + test_without_manual
    train = train_manual + train_without_manual

    # sanity checks
    assert(len(test) + len(train) == len(all_filenames))
    has_overlap = set(test_manual) & set(train_manual)
    assert(not has_overlap)
    has_overlap = set(test_without_manual) & set(train_without_manual)
    assert(not has_overlap)
    has_overlap = set(test_manual) & set(test_without_manual)
    assert(not has_overlap)
    has_overlap = set(train_manual) & set(train_without_manual)
    assert(not has_overlap)
    has_overlap = set(test) & set(train)
    assert(not has_overlap)

    # save result
    folder_to_filename_dict[folder] = {"test":test, "train":train}

C:\Users\isabe\Downloads\l2 artic dataset\BWC
Total number of files with manual annotations: 150
30 files with manual annotation in test and 120 in train
Total number of files: 1130
Total number of files without manual annotations: 980
196 files with no manual annotation in test and 784 in train
C:\Users\isabe\Downloads\l2 artic dataset\HJK
Total number of files with manual annotations: 150
30 files with manual annotation in test and 120 in train
Total number of files: 1131
Total number of files without manual annotations: 981
196 files with no manual annotation in test and 785 in train
C:\Users\isabe\Downloads\l2 artic dataset\HKK
Total number of files with manual annotations: 150
30 files with manual annotation in test and 120 in train
Total number of files: 1131
Total number of files without manual annotations: 981
196 files with no manual annotation in test and 785 in train
C:\Users\isabe\Downloads\l2 artic dataset\HQTV
Total number of files with manual annotations: 150
30 files wi

Copy the files into new folders based on their split

In [ ]:
folder_to_extension_mapping = {"wav": ".wav", "transcript":".txt", "textgrid":".TextGrid", "annotation":".TextGrid"}
for folder_path, split_details in folder_to_filename_dict.items():
    print(folder_path)
    test_filenames, train_filenames = split_details["test"], split_details["train"]
    train_split_folder_path = os.path.join(folder_path, "train").replace("l2 artic dataset", "l2 artic dataset split")
    for filename in train_filenames:
        for inner_folder, extension in folder_to_extension_mapping.items():
            # check if the file is present in src
            full_filename = filename+extension
            src = os.path.join(folder_path, inner_folder, full_filename)
            if os.path.exists(src):
                dest_folder = os.path.join(train_split_folder_path, inner_folder)
                dest = os.path.join(dest_folder, full_filename)
                if not os.path.exists(dest_folder):
                    os.makedirs(dest_folder)
                shutil.copyfile(src, dest)
    print("Train folder copying is complete")

    test_split_folder_path = os.path.join(folder_path, "test").replace("l2 artic dataset", "l2 artic dataset split")
    for filename in test_filenames:
        for inner_folder, extension in folder_to_extension_mapping.items():
            # check if the file is present in src
            full_filename = filename+extension
            src = os.path.join(folder_path, inner_folder, full_filename)
            if os.path.exists(src):
                dest_folder = os.path.join(test_split_folder_path, inner_folder)
                dest = os.path.join(dest_folder, full_filename)
                if not os.path.exists(dest_folder):
                    os.makedirs(dest_folder)
                shutil.copyfile(src, dest)
    print("Test folder copying is complete")

    # check that transcript, textgrid and annotations folder have same files
    for split_folder_path in [train_split_folder_path, test_split_folder_path]:
        transcript_files = glob.glob(os.path.join(split_folder_path, "transcript", "*.txt"))
        transcript_filenames = [path.split("\\")[-1][:-4] for path in transcript_files]
        textgrid_files = glob.glob(os.path.join(split_folder_path, "textgrid", "*.TextGrid"))
        textgrid_filenames = [path.split("\\")[-1][:-9] for path in textgrid_files]
        wav_files = glob.glob(os.path.join(split_folder_path, "wav", "*.wav"))
        wav_filenames = [path.split("\\")[-1][:-4] for path in wav_files]
        assert(len(transcript_filenames) == len(textgrid_filenames) == len(wav_filenames))
        assert(set(transcript_filenames) == set(textgrid_filenames) == set(wav_filenames))

    # check for any data loss
    all_train_files = [path.split("\\")[-1][:-4] for path in glob.glob(os.path.join(train_split_folder_path, "wav", "*.wav"))]
    assert(set(all_train_files) == set(train_filenames))
    all_test_files = [path.split("\\")[-1][:-4] for path in glob.glob(os.path.join(test_split_folder_path, "wav", "*.wav"))]
    assert(set(all_test_files) == set(test_filenames))

    # check the annations folder adds up to what is expected
    train_annotation_files = glob.glob(os.path.join(train_split_folder_path, "annotation", "*.TextGrid"))
    test_annotation_files = glob.glob(os.path.join(test_split_folder_path, "annotation", "*.TextGrid"))
    assert(len(train_annotation_files) == 120)
    assert(len(test_annotation_files) == 30)


C:\Users\isabe\Downloads\l2 artic dataset\BWC
Train folder copying is complete
Test folder copying is complete
set()
C:\Users\isabe\Downloads\l2 artic dataset\HJK
Train folder copying is complete
Test folder copying is complete
set()
C:\Users\isabe\Downloads\l2 artic dataset\HKK
Train folder copying is complete
Test folder copying is complete
set()
C:\Users\isabe\Downloads\l2 artic dataset\HQTV
Train folder copying is complete
Test folder copying is complete
set()
C:\Users\isabe\Downloads\l2 artic dataset\LXC
Train folder copying is complete
Test folder copying is complete
set()
C:\Users\isabe\Downloads\l2 artic dataset\NCC
Train folder copying is complete
Test folder copying is complete
set()
C:\Users\isabe\Downloads\l2 artic dataset\PNV
Train folder copying is complete
Test folder copying is complete
set()
C:\Users\isabe\Downloads\l2 artic dataset\THV
Train folder copying is complete
Test folder copying is complete
set()
C:\Users\isabe\Downloads\l2 artic dataset\TLV
Train folder copy